# The Basics of LangChain
open this [activeloop](https://learn.activeloop.ai/courses/take/langchain/multimedia/46317643-langchain-101-from-zero-to-hero) webpage for reference

In [6]:
# import the dotenv library to use API keys
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(),override=True)

True

### The LLMs

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(model="text-davinci-003", temperature=0.9)

prompt = """Suggest a personalized workout routine for someone 
        looking to improve cardiovascular endurance 
        and prefers outdoor activities."""
        
print(llm(prompt))

c:\Users\Veda Kashyap\anaconda3\envs\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.7.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(




Day 1: 
- 30 minutes of jogging
- 10 minutes of rest
- 10 minutes of high-intensity interval training (e.g. sprinting for 30 seconds, then jogging for 30 seconds, repeating for 10 minutes)

Day 2: 
- 20 minutes of walking or running on uneven terrain
- 10 minutes of dynamic stretching
- 10 minutes of skipping

Day 3: 
- 30 minutes of biking 
- 10 minutes of rest
- 10 minutes of sprinting 

Day 4: 
- 20 minutes of swimming
- 10 minutes of rest
- 10 minutes of jumping jacks

Day 5: 
- 20 minutes of rowing
- 10 minutes of rest
- 10 minutes of hill sprints


### The Chains

In [5]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))



EcoPure Bottles


### The Memory
Provides memory to store Conversation history. `ConversationBufferMemory` acts as a wrapper to `ChatMessageHistory`

In [7]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!
Human: What can yo

### DeepLake
Deeplake is a vector store and a datalake fused together.

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# instantiate the LLM and Embedding Models
llm=OpenAI(model='text-davinci-003',
           temperature=0)
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

# create our documents
texts=["Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=0)
docs= text_splitter.create_documents(texts)

# Create a deeplake dataset that can be later stored in vector store
activeloop_org_id = "vedakashyap"
activeloop_dataset_name = "langchain_course_zerotohero"

dataset_path = f"hub://{activeloop_org_id}/{activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path,
              embedding_function=embeddings)

db.add_documents(docs)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


/

Dataset(path='hub://vedakashyap/langchain_course_zerotohero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['0469c1a8-5f8d-11ee-b082-705a0f2a1e4a',
 '0469c1a9-5f8d-11ee-b8f9-705a0f2a1e4a']

In [14]:
# Since we have created a deeplake DB with our data
# The next step is to use the RetrievalQA chain

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()      
)

Now let's create an agent that uses the `RetrievalQA` chain as a tool

In [15]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools=[
    Tool(
        name="Retrieval QA system",
        func=retrieval_qa.run,
        description="Useful for retrieving the relevant contexts for QA"
    )
]

agent=initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Now we can ask the agent a question and see the magic happen
response = agent.run('When was Napoleone born?')
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA system
Action Input: Napoleone birth date
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.
